In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.layers import Dense
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [2]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful']

In [3]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

def load_data(test_size):
    x,y = [],[]
    for file in glob.glob("../input/ravdess-emotional-song-audio/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc = True, chroma = True, mel = True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size = test_size, random_state = 0) 

x_train, x_test, y_train, y_test = load_data(test_size = 0.2) 

In [4]:
y_train

['fearful',
 'fearful',
 'calm',
 'calm',
 'happy',
 'fearful',
 'calm',
 'happy',
 'happy',
 'fearful',
 'calm',
 'fearful',
 'fearful',
 'happy',
 'fearful',
 'calm',
 'fearful',
 'happy',
 'calm',
 'happy',
 'happy',
 'happy',
 'calm',
 'calm',
 'calm',
 'happy',
 'calm',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'calm',
 'calm',
 'fearful',
 'happy',
 'fearful',
 'happy',
 'fearful',
 'calm',
 'calm',
 'fearful',
 'calm',
 'fearful',
 'fearful',
 'calm',
 'calm',
 'fearful',
 'calm',
 'calm',
 'calm',
 'happy',
 'happy',
 'fearful',
 'fearful',
 'happy',
 'calm',
 'fearful',
 'fearful',
 'calm',
 'happy',
 'fearful',
 'happy',
 'happy',
 'happy',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'calm',
 'calm',
 'fearful',
 'calm',
 'calm',
 'calm',
 'fearful',
 'happy',
 'fearful',
 'calm',
 'fearful',
 'calm',
 'happy',
 'happy',
 'happy',
 'calm',
 'calm',
 'happy',
 'happy',
 'happy',
 'calm',
 'fearful',
 'calm',
 'happy',
 'fearful',

In [5]:
# print((x_train.shape[0], y_train.shape[0]))
ans = []

for i in y_test:
    if i not in ans:
        ans.append(i)
print(ans)

['fearful', 'happy', 'calm']


In [6]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

In [7]:
x_traincnn = np.expand_dims(x_train, axis=2)
x_testcnn = np.expand_dims(x_test, axis=2)

In [8]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [9]:
model = Sequential()

model.add(Conv1D(filters = 32, kernel_size = 5, activation = 'relu', input_shape = [180, 1]))
model.add(BatchNormalization())
model.add(MaxPool1D(pool_size = 8, strides = 4))
model.add(Dropout(0.25))

model.add(Conv1D(filters = 32, kernel_size = 5, activation = 'relu')) 
model.add(MaxPool1D(pool_size = 8, strides = 4))
model.add(Dropout(0.25))

model.add(Conv1D(filters = 64, kernel_size = 3, activation = 'relu')) 
model.add(MaxPool1D(pool_size = 8, strides = 3, padding = "same"))
model.add(Dropout(0.25))

model.add(Conv1D(filters = 128, kernel_size = 2, activation = 'relu', padding = "same")) 
model.add(MaxPool1D(pool_size = 8, strides = 4, padding = "same"))

model.add(Conv1D(filters = 256, kernel_size = 3, activation = 'relu', padding = "same")) 
model.add(MaxPool1D(pool_size = 8, strides = 2, padding = "same"))


model.add(Flatten())

model.add(Dense(100, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))  

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 176, 32)           192       
_________________________________________________________________
batch_normalization (BatchNo (None, 176, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 43, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 43, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39, 32)            5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 32)             0

In [11]:
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
cnnhistory = model.fit(x_traincnn, y_train, batch_size = 16, epochs = 700, validation_data = (x_testcnn, y_test)) 

Epoch 1/700
28/28 [==============================] - 1s 20ms/step - loss: 1.1152 - accuracy: 0.3084 - val_loss: 1.0907 - val_accuracy: 0.3423
Epoch 2/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0919 - accuracy: 0.3560 - val_loss: 1.0952 - val_accuracy: 0.3243
Epoch 3/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0726 - accuracy: 0.3787 - val_loss: 0.9868 - val_accuracy: 0.4595
Epoch 4/700
28/28 [==============================] - 0s 7ms/step - loss: 1.0485 - accuracy: 0.4422 - val_loss: 0.9855 - val_accuracy: 0.5135
Epoch 5/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0122 - accuracy: 0.4671 - val_loss: 0.9875 - val_accuracy: 0.5405
Epoch 6/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0029 - accuracy: 0.4671 - val_loss: 0.9520 - val_accuracy: 0.5225
Epoch 7/700
28/28 [==============================] - 0s 6ms/step - loss: 1.0062 - accuracy: 0.4898 - val_loss: 0.9489 - val_accuracy: 0.5405
Epoch 8/700
